In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from datetime import datetime

In [2]:
def dividir_columnas(df):
    columnas_transformadas = {}
    columnas_no_transformadas = []
    columnas_total = list(df.columns)
    for col in columnas_total:
        if col in list(correspondencias.keys()):
            columnas_transformadas[col] = correspondencias[col]
        else:
            columnas_no_transformadas.append(col)
    return (columnas_transformadas, columnas_no_transformadas)

def transformar_columna(cols_id, datos_finales, nulos, nombre_col, tipos, valor_minimo, valor_maximo, rango):
    if nombre_col in list(datos_finales.columns):
        for i in list(datos_finales.index):
            if datos_finales.loc[i,nombre_col] is None or pd.isnull(datos_finales.loc[i,nombre_col]):
                nulos = nulos.append({"dato_inicial": datos_finales.loc[i,nombre_col],
                              "motivo": "Nulo en csv",
                              "ID_columna_transformada": cols_id[nombre_col],
                              "PATIENT_ID": i}, ignore_index = True)
                datos_finales.loc[i,nombre_col] = "."
            else:
                if rango:
                    if not (isinstance(datos_finales.loc[i,nombre_col], tipos[0]) or isinstance(datos_finales.loc[i,nombre_col], tipos[1])) or datos_finales.loc[i,nombre_col]<=valor_minimo or datos_finales.loc[i,nombre_col]>=valor_maximo:
                        nulos = nulos.append({"dato_inicial": datos_finales.loc[i,nombre_col],
                                  "motivo": "Fuera de rango",
                                  "ID_columna_transformada": cols_id[nombre_col],
                                  "PATIENT_ID": i}, ignore_index = True)
                        datos_finales.loc[i,nombre_col] = "."
    else:
        datos_finales[nombre_col] = "."
        for i in list(datos_finales.index):
            nulos = nulos.append({"dato_inicial": ".",
                              "motivo": "Columna no aparece en csv",
                              "ID_columna_transformada": cols_id[nombre_col],
                              "PATIENT_ID": i}, ignore_index = True)
    return(datos_finales, nulos)

def transformar_datos(df_datos, cols_t, id_inicial):
    nulos = pd.DataFrame({"dato_inicial": [], "motivo": [], "ID_columna_transformada": [], "PATIENT_ID": []})
    
    datos_finales = pd.DataFrame(df_datos[list(cols_t.keys())])
    datos_finales.index = list(range(id_inicial, id_inicial + len(df_datos)))
    datos_finales = datos_finales.rename(columns = cols_t)
    datos_finales["DMRGENDR"] = datos_finales["DMRGENDR"].apply(lambda x: 1 if x == "FEMALE" else 0 if x == "MALE" else x)
    datos_finales["DSXOS"] = datos_finales["DSXOS"].apply(lambda x: 0 if x == "Domicilio" else 1 if x == "Fallecimiento" else 2 if x == "Traslado al Hospital" else x)
    datos_finales["CSXBTPA"] = datos_finales["CSXBTPA"].apply(lambda x: float(x.replace(",", ".")))
    datos_finales["CSXBTPHn"] = datos_finales["CSXBTPHn"].apply(lambda x: float(x.replace(",", ".")))
    
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    cols_id = {}
    sql_id_cols = 'SELECT ID_columna_transformada FROM columnas_transformadas WHERE nombre_final = %s'
    for col in list(datos_finales.columns):
        cursor.execute(sql_id_cols, [col])
        ids = cursor.fetchall()
        id_col = ids[0][0]
        cols_id[col] = id_col
    cursor.close()
    conn.close()
    
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"DMRAGEYR", (np.int64, int), 0, 150, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"DMRGENDR", (np.int64, int), -1, 2, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"DATAD", (np.int64, int), 0, 0, False)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"DATLGTI", (np.float64, float), 0, 180, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"DATDSI", (np.int64, int), 0, 0, False)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"DSXOS", (np.int64, int), -1, 3, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"CSXBTPA", (np.float64, float), 25, 43, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"CSXCHRA", (np.int64, int), 0, 288, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"LBXSGLA", (np.int64, int), 33, 559, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"CSXOSTA", (np.int64, int), 0, 100, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"CSXSYA", (np.int64, int), 74, 270, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"CSXDIA", (np.int64, int), 0, 124, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"CSXCHRHn", (np.int64, int), 0, 288, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"CSXBTPHn", (np.float64, float), 25, 43, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"LBXSGLHn", (np.int64, int), 33, 559, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"CSXOSTHn", (np.int64, int), 0, 100, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"CSXSYHn", (np.int64, int), 74, 270, True)
    (datos_finales, nulos) = transformar_columna(cols_id, datos_finales, nulos,"CSXDIHn", (np.int64, int), 0, 124, True)
    
    nulos.fillna("Nulo", inplace = True)
    return (datos_finales, nulos)

def subir_df_a_tabla(df, nombre_tabla):
    sql = "INSERT INTO " + nombre_tabla +" ("
    for col in list(df.columns):
        sql += col + ", "
    sql = sql[:-2] + ") VALUES (" + "%s, " * len(df.columns)
    sql = sql[:-2] + ")"
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    for i in range(len(df)):
        cursor.execute(sql, list(df.iloc[i]))
        conn.commit()
    cursor.close()
    conn.close()
    
def cargar_datos_finales(df, nulos):
    df = df.applymap(lambda x: str(x))
    subir_df_a_tabla(df, "datos_finales")
    subir_df_a_tabla(nulos, "nulos")

    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    sql_PATIENT_IDs = "SELECT PATIENT_ID FROM datos_finales ORDER BY PATIENT_ID DESC"
    cursor.execute(sql_PATIENT_IDs)
    ids = cursor.fetchall()
    id_final = ids[0][0]
    id_inicial = id_final - len(df) + 1
    cursor.close()
    conn.close()
    return id_inicial

def iniciar_carga(nombre_archivo_procedente):
    fecha = str(datetime.now())[:-7]
    df_carga = pd.DataFrame({"fecha": [fecha],
                             "nombre_archivo_procedente": [nombre_archivo_procedente]})
    subir_df_a_tabla(df_carga, "cargas")
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    sql_id_carga = "SELECT ID_carga FROM cargas WHERE fecha = %s"
    cursor.execute(sql_id_carga, [fecha])
    id_carga =cursor.fetchone()[0]
    cursor.close()
    conn.close()
    return id_carga

def carga_columnas_transformadas(cols_t, id_carga):
    df_cols_transformadas = pd.DataFrame({"nombre_original": list(cols_t.keys()),
                                      "nombre_final": list(cols_t.values()),
                                     "ID_carga": str(id_carga)})
    subir_df_a_tabla(df_cols_transformadas, "columnas_transformadas")
    
def carga_columnas_no_transformadas(cols_n, id_carga):
    df_cols_no_transformadas = pd.DataFrame({"nombre": cols_n,
                                     "ID_carga": str(id_carga)})
    subir_df_a_tabla(df_cols_no_transformadas, "columnas_no_transformadas")

def preparar_datos_no_cargados(df_datos, cols_n, id_inicial):
    datos_no_cargados = pd.DataFrame(df_datos[cols_n])
    datos_no_cargados.index = list(range(id_inicial, id_inicial + len(df_datos)))
    
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    cols_n_id = {}
    sql_id_col_n = 'SELECT ID_columna_no_transformada FROM columnas_no_transformadas WHERE nombre = %s'
    for col in list(datos_no_cargados.columns):
        cursor.execute(sql_id_col_n, [col])
        
        ids = cursor.fetchall()
        id_col_n = ids[0][0]
        cols_n_id[col] = id_col_n
    cursor.close()
    conn.close()
    datos_no_cargados = datos_no_cargados.rename(columns = cols_n_id)
    return datos_no_cargados

def cargar_datos_no_cargados(df_datos_no_cargados):
    datos_subir = pd.DataFrame({"valor":[],
                                "ID_columna_no_transformada":[],
                                "PATIENT_ID":[]})
    for i in range(len(df_datos_no_cargados)):
        for j in range(len(df_datos_no_cargados.iloc[0])):
            datos_subir = datos_subir.append({"valor": str(df_datos_no_cargados.iloc[i,j]),
                                              "ID_columna_no_transformada": str(df_datos_no_cargados.columns[j]),
                                              "PATIENT_ID": str(df_datos_no_cargados.index[i])}, ignore_index = True)
    subir_df_a_tabla(datos_subir, "datos_no_cargados")    

def obtener_col_transf(id_carga):
    sql = "SELECT nombre_original, nombre_final FROM columnas_transformadas WHERE ID_carga = %s"
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    cursor.execute(sql, [id_carga])
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    return data

def obtener_col_no_transf(id_carga):
    sql = "SELECT nombre FROM columnas_no_transformadas WHERE ID_carga = %s"
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    cursor.execute(sql, [id_carga])
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    return data

def obtener_no_cargados(id_carga):
    sql = "SELECT d.valor, d.PATIENT_ID, c.nombre FROM columnas_no_transformadas as c, datos_no_cargados as d WHERE c.ID_columna_no_transformada = d.ID_columna_no_transformada AND ID_carga = %s"
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    cursor.execute(sql, [id_carga])
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    return data

In [3]:
def cargar(nombre_archivo_procedente): #Funcion final
    datos = pd.read_csv(nombre_archivo_procedente, sep = ";")
    (columnas_transformadas, columnas_no_transformadas) = dividir_columnas(datos)
    
    id_carga = iniciar_carga(nombre_archivo_procedente)
    carga_columnas_transformadas(columnas_transformadas, id_carga)
    
    (datos_finales, nulos) = transformar_datos(datos, columnas_transformadas, 1)
    id_inicial = cargar_datos_finales(datos_finales, nulos)
    
    carga_columnas_no_transformadas(columnas_no_transformadas, id_carga)
    datos_no_cargados = preparar_datos_no_cargados(datos, columnas_no_transformadas, id_inicial)
    cargar_datos_no_cargados(datos_no_cargados)
    print(f"Datos cargados correctamente.\nIds pacientes nuevos: {id_inicial}-{id_inicial + len(datos) -1}.\nid_carga = {id_carga}.")
    
def restaurar_datos_no_cargados(id_carga):
    datos_no_cargados = pd.DataFrame(obtener_no_cargados(id_carga))
    idx = datos_no_cargados[1].unique()
    cols = datos_no_cargados[2].unique()
    df_datos_no_cargados = pd.DataFrame(np.zeros((len(idx),len(cols))), columns = cols, index = idx)
    for i in range(len(datos_no_cargados)):
        df_datos_no_cargados.loc[datos_no_cargados[1][i], datos_no_cargados[2][i]]= datos_no_cargados[0][i]
    return df_datos_no_cargados

In [4]:
correspondencias = {
    "EDAD/AGE": "DMRAGEYR",
    "SEXO/SEX": "DMRGENDR",
    "F_INGRESO/ADMISSION_D_ING/INPAT": "DATAD",
    "UCI_DIAS/ICU_DAYS": "DATLGTI",
    "F_ALTA/DISCHARGE_DATE_ING": "DATDSI",
    "MOTIVO_ALTA/DESTINY_DISCHARGE_ING": "DSXOS",  
    #"F_INGRESO/ADMISSION_DATE_URG/EMERG": "", # No sale 
    #"HORA/TIME_ADMISION/ADMISSION_URG/EMERG": "",  # No sale 
    #"ESPECIALIDAD/DEPARTMENT_URG/EMERG": "",   # No sale 
    #"DIAG_URG/EMERG": "",   # No sale 
    #"DESTINO/DESTINY_URG/EMERG": "",  # No sale 
    "TEMP_PRIMERA/FIRST_URG/EMERG": "CSXBTPA",
    "FC/HR_PRIMERA/FIRST_URG/EMERG": "CSXCHRA",
    "GLU_PRIMERA/FIRST_URG/EMERG": "LBXSGLA",
    "SAT_02_PRIMERA/FIRST_URG/EMERG": "CSXOSTA",
    "TA_MAX_PRIMERA/FIRST/EMERG_URG": "CSXSYA",
    "TA_MIN_PRIMERA/FIRST_URG/EMERG": "CSXDIA",
    #"HORA/TIME_CONSTANT_ULTIMA/LAST_URG/EMERG": "",   # No sale
    "FC/HR_ULTIMA/LAST_URG/EMERG": "CSXCHRHn",
    "TEMP_ULTIMA/LAST_URG/EMERG": "CSXBTPHn",
    "GLU_ULTIMA/LAST_URG/EMERG": "LBXSGLHn",
    "SAT_02_ULTIMA/LAST_URG/EMERG": "CSXOSTHn",
    "TA_MAX_ULTIMA/LAST_URGEMERG": "CSXSYHn",
    "TA_MIN_ULTIMA/LAST_URG/EMERG": "CSXDIHn"
}
config = {'host': "localhost",
          'port': 3306,
          'user': "covid_user",
          'passwd': "covid_password",
          'db': "datoscovid"}

In [5]:
cargar("ejemplo-datos.csv")

Datos cargados correctamente.
Ids pacientes nuevos: 1-68.
id_carga = 1.


In [6]:
restaurar_datos_no_cargados(1)

,PATIENT ID,F_INGRESO/ADMISSION_DATE_URG/EMERG,HORA/TIME_ADMISION/ADMISSION_URG/EMERG,ESPECIALIDAD/DEPARTMENT_URG/EMERG,DIAG_URG/EMERG,DESTINO/DESTINY_URG/EMERG,HORA/TIME_CONSTANT_ULTIMA/LAST_URG/EMERG
1,1.0,06/03/2020,4:02:00,Medicina de Urgencias,DIFICULTAD RESPIRATORIA,Ingreso,5:28:00
2,nan,06/03/2020,10:56:00,Medicina de Urgencias,DIFICULTAD RESPIRATORIA,Ingreso,11:18:00
3,nan,06/03/2020,16:14:00,Medicina Interna,FIEBRE,Ingreso,18:18:00
4,nan,07/03/2020,9:16:00,Medicina de Urgencias,FIEBRE,Ingreso,9:27:00
5,nan,07/03/2020,16:57:00,Medicina de Urgencias,MALESTAR,Ingreso,18:12:00
...,...,...,...,...,...,...,...
64,nan,13/03/2020,19:57:00,Medicina de Urgencias,DIFICULTAD RESPIRATORIA,Ingreso,20:06:00
65,nan,18/03/2020,18:02:00,Medicina de Urgencias,DIFICULTAD RESPIRATORIA,Ingreso,21:53:00
66,nan,18/03/2020,0:00:00,Medicina de Urgencias,DIFICULTAD RESPIRATORIA,Ingreso,0:05:00
67,nan,18/03/2020,14:45:00,Medicina de Urgencias,DIFICULTAD RESPIRATORIA,Ingreso,15:06:00


In [7]:
def informe_carga(id_carga):
    print(f"Informe de Carga {id_carga}:\n")
    cols_todo_nulos = list(correspondencias.values())
    cols_t = obtener_col_transf(id_carga)
    print(f"Un total de {len(cols_t)} variables han sido armonizadas con éxito:")
    for col in cols_t:
        print(f"- {col[0]} -> {col[1]}")
        cols_todo_nulos.remove(col[1])
    print("\n")
    cols_n = obtener_col_no_transf(id_carga)
    print(f"Un total de {len(cols_n)} variables no se han podido armonizar:")
    for col in cols_n:
        print(f"- {col[0]}")
    print("\n")
    print(f"Un total de {len(cols_todo_nulos)} variables están en la BD pero no aparecían en el csv, por lo que se han rellenado con nulos:")
    for col in cols_todo_nulos:
        print(f"- {col[0]}")
    print("\n")

In [8]:
informe_carga(1)

Informe de Carga 1:

Un total de 18 variables han sido armonizadas con éxito:
- EDAD/AGE -> DMRAGEYR
- SEXO/SEX -> DMRGENDR
- F_INGRESO/ADMISSION_D_ING/INPAT -> DATAD
- UCI_DIAS/ICU_DAYS -> DATLGTI
- F_ALTA/DISCHARGE_DATE_ING -> DATDSI
- MOTIVO_ALTA/DESTINY_DISCHARGE_ING -> DSXOS
- TEMP_PRIMERA/FIRST_URG/EMERG -> CSXBTPA
- FC/HR_PRIMERA/FIRST_URG/EMERG -> CSXCHRA
- GLU_PRIMERA/FIRST_URG/EMERG -> LBXSGLA
- SAT_02_PRIMERA/FIRST_URG/EMERG -> CSXOSTA
- TA_MAX_PRIMERA/FIRST/EMERG_URG -> CSXSYA
- TA_MIN_PRIMERA/FIRST_URG/EMERG -> CSXDIA
- FC/HR_ULTIMA/LAST_URG/EMERG -> CSXCHRHn
- TEMP_ULTIMA/LAST_URG/EMERG -> CSXBTPHn
- GLU_ULTIMA/LAST_URG/EMERG -> LBXSGLHn
- SAT_02_ULTIMA/LAST_URG/EMERG -> CSXOSTHn
- TA_MAX_ULTIMA/LAST_URGEMERG -> CSXSYHn
- TA_MIN_ULTIMA/LAST_URG/EMERG -> CSXDIHn


Un total de 7 variables no se han podido armonizar:
- PATIENT ID
- F_INGRESO/ADMISSION_DATE_URG/EMERG
- HORA/TIME_ADMISION/ADMISSION_URG/EMERG
- ESPECIALIDAD/DEPARTMENT_URG/EMERG
- DIAG_URG/EMERG
- DESTINO/DESTIN